# Sector Transition Report Demo

This tool creates a short report on sectoral transition in given country and sector, answering key questions relevant to ASCOR analysts

        Input: country and sector

        Output: one paragraph answer to key questions

The report follows a pre-defined, binding strucutre, answering these two questions:

1. Has this sector in this country set a net zero target?
2. (CHALLENGING) How has the climate legislation of this country in this sector changed in the past 50 years? 

Because of the complexity of the latter, it is broken into time periods, each evaluated by a different LLM and summarized.

It is finally compiled into markdown with enforced format

### 0. Setup


In [1]:
# Import necessary modules
import sys
import os
from pathlib import Path

# Get the absolute path of the project root directory
notebook_dir = Path(os.getcwd())  
project_root = notebook_dir.parent.parent  # Go up TWO levels instead of one

# Add project root to Python path
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))
    print(f"Added {project_root} to sys.path")


from scripts.climate_policy_pipelines.sectoral_analysis.pipeline import (
    generate_ascor_report, 
    format_report_as_markdown
)

from scripts.climate_policy_pipelines.sectoral_analysis.classes import (
    NetZeroTargetDetails,
    NetZeroAssessment,
    PeriodAnalysis1970s,
    PeriodAnalysis1990s,
    PeriodAnalysis2010s,
    LegislativeTimeline,
    LegislativeSummary,
    SimpleASCORReport
)

Added /Users/jessiefung/Desktop/DS205/group-6-final-project to sys.path


/Users/jessiefung/Desktop/DS205/group-6-final-project/.venv/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## 1. Intended Structre

The report should follows an enforced markdown strucutre

```ASCOR Sectoral Transition Report
Executive Summary
Country: [COUNTRY NAME]
Sector: [SECTOR NAME]
Key Findings
1. Net Zero Target Assessment
Question: Have they set a net zero target?

[One paragraph response analyzing whether the specified country has established net zero commitments for the given sector. Include timeline details, scope of coverage, and any conditional aspects of the target.]

Key Details:

Target Date: [DATE or "Not Set"]
Scope: [Economy-wide/Sectoral/Conditional]
Legal Status: [Legally Binding/Policy Commitment/Aspirational]
Confidence in Assessment:

2. Legislative Evolution Assessment
Question: How has the legislation of this country changed in the past 50 years?

[One paragraph response analyzing the evolution of climate and sectoral legislation over the past five decades. Focus on major policy shifts, regulatory frameworks, and legislative milestones relevant to the specified sector.]

Legislative Timeline:

1970s-1980s: [Key developments]
1990s-2000s: [Key developments]
2010s-Present: [Key developments]
Confidence in Assessment:
```

The strucure is enforced using classes which are substituted into the final markdown scheme. For example this is the class that must be follows for legislative summaries:

In [2]:
from typing_extensions import Annotated, TypedDict
from typing import Literal

class LegislativeSummary(TypedDict):
    """Legislative evolution summary from synthesis LLM"""
    legislation_response: Annotated[str, ..., "One paragraph analyzing 50 years of legislative evolution, focusing on policy shifts and regulatory frameworks"]
    legislation_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the legislative assessment"]

These are then substituted into the report strucutre `SimpleASCORReport`

In [3]:
class SimpleASCORReport(TypedDict):
    """Final assembled report structure"""
    
    # Executive Summary fields
    country: Annotated[str, ..., "Name of the country being analyzed"]
    sector: Annotated[str, ..., "Name of the sector being analyzed"]
    
    # Net Zero Target Assessment (from specialized LLM)
    net_zero_response: Annotated[str, ..., "One paragraph analyzing net zero commitments"]
    net_zero_details: Annotated[NetZeroTargetDetails, ..., "Structured key details about the net zero target"]
    net_zero_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the net zero assessment"]
    
    # Legislative Evolution Assessment (assembled from multiple LLMs)
    legislation_response: Annotated[str, ..., "One paragraph analyzing 50 years of legislative evolution"]
    legislative_timeline: Annotated[LegislativeTimeline, ..., "Structured timeline of legislative changes by period"]
    legislation_confidence: Annotated[Literal["High", "Medium", "Low"], ..., "Confidence level in the legislative assessment"]

## 2. Demo

Lets see how it runs for the power sector in Albania:

In [4]:
country = "ALB"
sector = "Power"

report = generate_ascor_report(country=country, sector=sector)

markdown_report = format_report_as_markdown(report)
display(Markdown(markdown_report))

LengthFinishReasonError: Could not parse response content as the length limit was reached - CompletionUsage(completion_tokens=8192, prompt_tokens=1984, total_tokens=10176, completion_tokens_details=None, prompt_tokens_details=None)

**Warning**

Sometimes reports get too long - we are having trouble fixing this bug